Alejandro Castro Arévalo-A01636235
ML Tutorial 1

# 1 Preparar Data

# 1.1 Leer Dataset

In [ ]:
Input(x) -> Comentarios(review)
Output(y) -> Sentimientos

In [2]:
import pandas as pd

df_review = pd.read_csv('IMDB Dataset.csv')
df_review

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
df_review.value_counts('sentiment')

sentiment
negative    25000
positive    25000
dtype: int64

In [4]:
df_positivo = df_review[df_review['sentiment']=='positive'][:9000]
df_negativo = df_review[df_review['sentiment']=='negative'][:1000]

df_review_des = pd.concat([df_positivo, df_negativo])
df_review_des.value_counts('sentiment')

sentiment
positive    9000
negative    1000
dtype: int64

# 1.2 Dataset Desbalanceado

In [9]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
#rus.fit_resample(df_review_des[['review']], df_review_des['sentiment'])
df_review_bal, df_review_bal['sentiment'] = rus.fit_resample(df_review_des[['review']],
                                                          df_review_des['sentiment'])
df_review_bal.value_counts(['sentiment'])

sentiment
negative     1000
positive     1000
dtype: int64

# 1.3 Separando Data para Entrenar y Testear

In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_review_bal, test_size=0.33, random_state=42)

In [14]:
train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

# 2 Representación de Texto (Bag of Words)

- CountVectorizer
- Tfidf

# 2.1 CountVectorizer

In [19]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
text = ["Amo escribir codigo en Python. Amo el código en Python",
        "Odio escribir codigo en Java. Odio el código en Java"]

df = pd.DataFrame({'review': ['review1', 'review2'], 'text':text})
cv = CountVectorizer()
cv_matrix = cv.fit_transform(df['text'])
df_dtm = pd.DataFrame(cv_matrix.toarray(), index=df['review'].values, columns=cv.get_feature_names_out())
df_dtm

,amo,codigo,código,el,en,escribir,java,odio,python
review1,2,1,1,1,2,1,0,0,2
review2,0,1,1,1,2,1,2,2,0


# 2.2 Tfidf (term frequency-inverse document frequency)

In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
text = ["Amo escribir codigo en Python. Amo el código en Python",
        "Odio escribir codigo en Java. Odio el código en Java"]

df = pd.DataFrame({'review': ['review1', 'review2'], 'text':text})
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['text'])
df_dtm = pd.DataFrame(tfidf_matrix.toarray(), index=df['review'].values, columns=tfidf.get_feature_names_out())
df_dtm

,amo,codigo,código,el,en,escribir,java,odio,python
review1,0.576152,0.204969,0.204969,0.204969,0.409937,0.204969,0.000000,0.000000,0.576152
review2,0.000000,0.204969,0.204969,0.204969,0.409937,0.204969,0.576152,0.576152,0.000000


# 2.3 Transformar Text Data a Numeric Data

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(train_x) #Obtener mejores parametros con fit y transformarlos
test_x_vector = tfidf.transform(test_x) #se aplica transformacion tambien a la data de testeo

In [23]:
train_x_vector #Se guarda/transforma en una matriz

<1340x20299 sparse matrix of type '<class 'numpy.float64'>'
	with 116275 stored elements in Compressed Sparse Row format>

# 3 Selección del Modelo

ML algoritmos

1. Aprendizaje supervisado (Supervised learning): Regresion (output numerico), Clasificacion (output discreto)
- Input: Review
- Output: Sentiment (discrete)
2. Aprendizaje no supervisado

# 3.1 Support Vector Machines (SVM)

In [25]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')
svc.fit(train_x_vector, train_y) #Muestra "SVC" que es que el modelo ha sido entrenado

SVC(kernel='linear')

## 3.1.1 Testeo

In [26]:
print(svc.predict(tfidf.transform(['A good movie']))) #buena pelicula
print(svc.predict(tfidf.transform(['An excellent movie']))) #excelente pelicula
print(svc.predict(tfidf.transform(['"I did not like this movie at all I gave this movie away"'])))# no gusto

['positive']
['positive']
['negative']


# 3.2 Decision Tree

In [27]:
from sklearn.tree import DecisionTreeClassifier

dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, train_y)

DecisionTreeClassifier()

# 3.3 Naive Bayes

In [28]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(train_x_vector.toarray(), train_y) #Primer atributo debe ser array, idk why

GaussianNB()

# 3.4 Logistic Regression

In [29]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_x_vector, train_y)

LogisticRegression()

# 4 Evaluación del Modelo

## 4.1 Precisión del modelo (Score)

In [32]:
# shift+tab para definición de función
print(svc.score(test_x_vector, test_y)) #Usamos data de testeo porque ya le aplicamos la transformación
print(dec_tree.score(test_x_vector, test_y))
print(gnb.score(test_x_vector.toarray(), test_y)) #Again, x debe ser array
print(lr.score(test_x_vector, test_y))

0.843939393939394
0.6954545454545454
0.6212121212121212
0.8318181818181818


## 4.2 F1 Score

F1 Score = 2(Recall Precision) / (Recall + Precision)

In [33]:
from sklearn.metrics import f1_score
#Explicación al min.58
#F1 Score tiene valores entre 0-1
f1_score(test_y, svc.predict(test_x_vector),
         labels=['positive', 'negative'],
         average=None)

array([0.85007278, 0.83728278])

## 4.3 Reporte de Clasificación

In [34]:
from sklearn.metrics import classification_report

print(classification_report(test_y, svc.predict(test_x_vector),
                            labels=['positive', 'negative']))

              precision    recall  f1-score   support

    positive       0.83      0.87      0.85       335
    negative       0.86      0.82      0.84       325

    accuracy                           0.84       660
   macro avg       0.84      0.84      0.84       660
weighted avg       0.84      0.84      0.84       660



## 4.4 Confusion Matrix

In [35]:
from sklearn.metrics import confusion_matrix
#Igual que el report pero con CM
print(confusion_matrix(test_y, svc.predict(test_x_vector),
                       labels=['positive', 'negative']))

[[292  43]
 [ 60 265]]


# 5 Optimización

# 5.1 GridSearchCV

In [36]:
from sklearn.model_selection import GridSearchCV
#C: Parametro de penalización
#kernel: Parte del sistema encargada del procesamiento
parametros = {'C':[1,4,8,16,32], 'kernel':['linear', 'rbf']}

svc = SVC()
svc_grid = GridSearchCV(svc, parametros, cv=5) #cv: CrossValidation
svc_grid.fit(train_x_vector, train_y) #Correr esto toma un rato

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 4, 8, 16, 32], 'kernel': ['linear', 'rbf']})

In [37]:
print(svc_grid.best_estimator_)
print(svc_grid.best_params_)

SVC(C=4)
{'C': 4, 'kernel': 'rbf'}


In [38]:
svc_grid.best_score_

0.8253731343283583